In [1]:
import copy
import pandas as pd

In [2]:
common_words_file_name = 'clean-3000-most-common.txt'
all_words_file_name = 'clean-words-raw.txt'
test_set_file_name = 'test-set.txt'
# load words
with open(f'data/{common_words_file_name}') as f:
    common_words = f.read().split('\n')
with open(f'data/{all_words_file_name}') as f:
    all_words = f.read().split('\n')
with open(f'data/{test_set_file_name}') as f:
    test_set = f.read().split('\n')

In [3]:
class Feedback:
    def __init__(self):
        self.correct = [None, None, None, None, None]
        self.wrong_order = []
        self.incorrect = []
    
    def add_incorrect_char(self, incorrect_char):
        self.incorrect.append(incorrect_char)

    def add_correct_char(self, correct_char, idx):
        self.correct[idx] = correct_char

    def add_wrong_order(self, wrong_order_char, idx):
        self.wrong_order.append([wrong_order_char, idx])

    def print(self):
        print(f'correct: {self.correct}')
        print(f'incorrect: {self.incorrect}')
        print(f'wrong order: {self.wrong_order}')

    def copy(self):
        return copy.deepcopy(self)

In [4]:
def evaluate_guess(guess, right_answer, feedback):
    """
    Knowing the right answer and given a guess and a feedback object
    return the expanded feedback for that guess
    """
    for idx, guess_char in enumerate(guess):
        # Find incorrect characters
        if guess_char not in right_answer:
            feedback.add_incorrect_char(guess_char)
        else:
            # Find if the character is in the right order
            if right_answer[idx] == guess_char:
                feedback.add_correct_char(guess_char, idx)
            else:
                feedback.add_wrong_order(guess_char, idx)
    return feedback

In [5]:
def get_possible_answers(list_of_words, feedback):
    "given the feedback, what are the possible answers?"
    # Filter out incorrect characters
    for incorrect_char in feedback.incorrect:
        list_of_words = [w for w in list_of_words if incorrect_char not in w]

    # Filter out correct characters
    for idx, correct_char in enumerate(feedback.correct):
        if correct_char:
            list_of_words = [w for w in list_of_words if w[idx] == correct_char]

    # Filter out the right characters but wrong order
    for [wrong_order_char, idx] in feedback.wrong_order:
        if wrong_order_char:
            list_of_words = [w for w in list_of_words if wrong_order_char in w and w[idx] != wrong_order_char]

    return list_of_words

In [6]:
def rank_options(list_of_words, feedback):
    """
    Ranks a list of words to find the best guess.
    If I chose this word, how long would the next list of possible words be?
    Takes a list of possible words (list) and a feedback object (correct, wrong order and incorrect characters)
    """
    matrix = [[None for idx in range(len(list_of_words))] for idx in range(len(list_of_words))]
    
    ranked_words = []
    for idx_guess, guess in enumerate(list_of_words):
        next_list_length = []
        for idx_right_answer, right_answer in enumerate(list_of_words):
            internal_feedback = feedback.copy()
            internal_feedback = evaluate_guess(guess, right_answer, internal_feedback)
            possible_answers = get_possible_answers(list_of_words, internal_feedback)
            matrix[idx_right_answer][idx_guess] = len(possible_answers)
            next_list_length.append(len(possible_answers))
        ranked_words.append([guess, sum(next_list_length) / len(next_list_length) * 1000])
    ranked_words.sort(key=lambda x: x[1])
    if len(list_of_words) < 20:
        df = pd.DataFrame(matrix, columns=list_of_words, index=list_of_words)
        print(df)
    else:
        print('Matrix is too large to print')
    return ranked_words

In [8]:
options = [1,2,3,4]

In [12]:
options[-1]

4

In [22]:
def print_options(options):
    bar_length = 30
    number_of_options_to_print = 20
    options_to_print = options[0: number_of_options_to_print]
    max_score = options_to_print[0][1]
    min_score = options_to_print[-1][1]
    score_range = max_score - min_score
    score_scale = bar_length / (max_score - min_score)
    print(min_score)
    print(max_score)
    print(score_scale)
    for [word, score] in options_to_print:
        bar = ''.join(['-' if (score - min_score) * score_scale >= idx else ' ' for idx in range(bar_length)])
        print(f'{bar} {word}')
    

In [23]:
options = [
    ['this', 40],
    ['that', 333],
    ['ptjer', 1000]
]
print_options(options)

1000
40
-0.03125
------------------------------ this
---------------------          that
-                              ptjer


# Interactive play

In [24]:
feedback = Feedback()
win = False
turn = 1
guess = 'arise'
while not win:

    print('')
    print('')
    print('------------------------------------------------')
    print(f'Round {turn}')
    print('')
    
    # Recommend option and ask for feedback from the user
    print(f'Try "{guess}"')
    guess = input("what word did you enter? ")
    colours = ''
    while len(colours) != 5:
        colours = input("what was the feedback? [g/b/y] ")
    if colours == 'ggggg':
        win = True
        print("Congratulations!")
    else:
        for idx, colour in enumerate(colours):
            if colour == 'g':
                # right guess 
                feedback.add_correct_char(guess[idx], idx)
            elif colour == 'y':
                # wrong order
                feedback.add_wrong_order(guess[idx], idx)
            elif colour == 'b':
                # wrong character
                feedback.add_incorrect_char(guess[idx])

        # Evaluate feedback and get new answers
        possible_answers = get_possible_answers(all_words, feedback)
#         possible_answers = get_possible_answers(common_words, feedback)
#         if len(possible_answers) < 3:
#             possible_answers = get_possible_answers(all_words, feedback)
        options = rank_options(possible_answers, feedback)
        print_options(options)
#         if len(options) < 1000:
#             print(options)
        guess = options[0][0]
        turn += 1
    



------------------------------------------------
Round 1

Try "arise"
what word did you enter? ridge
what was the feedback? [g/b/y] bybbb
Matrix is too large to print
14712.765957446809
9430.851063829787
-0.005679758308157099
------------------------------ saint
-----------------------------  tails
--------------------------     nails
-------------------------      satin
----------------------         antis
----------------------         snail
----------------------         slain
-------------------            stain
-----------                    saith
-----------                    pains
---------                      toils
--------                       mains
--------                       pails
-------                        hails
-------                        thais
----                           mails
---                            spilt
---                            laius
--                             baits
-                              loins


------------------------------

IndexError: list index out of range

# Integrate selenium

In [28]:
# Parsing
from selenium import webdriver
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import time

In [15]:
browser = webdriver.Chrome(ChromeDriverManager().install())
browser.get('https://www.powerlanguage.co.uk/wordle/')



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/franciscogalarza/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache
/var/folders/_0/3cmjzq_s335284nqqhkx35qw0000gn/T/ipykernel_23645/1184884865.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())


In [34]:
# Close pop up by clicking something
body = browser.find_element(by='xpath', value='//html')
body.click()

In [29]:
def write_wrod(word):
    for character in word:
        body.send_keys(character)
        time.sleep(0.3)
    body.send_keys(u'\ue007')

def get_state():
    

In [48]:
browser.find_element(by='id', value='board')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="board"]"}
  (Session info: chrome=97.0.4692.99)
Stacktrace:
0   chromedriver                        0x0000000102618e69 chromedriver + 5160553
1   chromedriver                        0x00000001025a3593 chromedriver + 4679059
2   chromedriver                        0x00000001021572c8 chromedriver + 172744
3   chromedriver                        0x000000010218cb62 chromedriver + 392034
4   chromedriver                        0x000000010218cd21 chromedriver + 392481
5   chromedriver                        0x00000001021bf304 chromedriver + 598788
6   chromedriver                        0x00000001021aa3fd chromedriver + 513021
7   chromedriver                        0x00000001021bd0ab chromedriver + 589995
8   chromedriver                        0x00000001021aa623 chromedriver + 513571
9   chromedriver                        0x000000010217fdce chromedriver + 339406
10  chrome

In [33]:
# write_wrod('arise')
body.send_keys(u'\ue007')

# Test dictionary

In [7]:
from PyDictionary import PyDictionary

In [8]:
dictionary=PyDictionary()

In [9]:
dictionary.meaning('cat')

{'Noun': ['feline mammal usually having thick soft fur and no ability to roar: domestic cats; wildcats',
  'an informal term for a youth or man',
  'a spiteful woman gossip',
  'the leaves of the shrub Catha edulis which are chewed like tobacco or used to make tea; has the effect of a euphoric stimulant',
  'a whip with nine knotted cords',
  'a large tracked vehicle that is propelled by two endless metal belts; frequently used for moving earth in construction and farm work',
  'any of several large cats typically able to roar and living in the wild',
  'a method of examining body organs by scanning them with X rays and using a computer to construct a series of cross-sectional scans along a single axis'],
 'Verb': ["beat with a cat-o'-nine-tails",
  'eject the contents of the stomach through the mouth']}

In [10]:
dictionary.meaning('whose')

Error: The Following Error occured: list index out of range


## This dictionary does not contain all words so it will not be useful.
#### Try this: https://stackoverflow.com/questions/3788870/how-to-check-if-a-word-is-an-english-word-with-python

In [11]:
# !pip install pyenchant

In [12]:
# !brew install enchant

In [13]:
import enchant

In [24]:
dictionary = enchant.Dict('en_GB')

In [25]:
dictionary.check('pizza')

True

In [8]:
# Helper
feedback = Feedback()

# Round 0
guess = 'arise'
feedback.add_incorrect_char('a')
feedback.add_incorrect_char('r')
# feedback.add_incorrect_char('i')
feedback.add_incorrect_char('s')
feedback.add_incorrect_char('e')
# feedback.add_correct_char('a', 0)
# feedback.add_correct_char('r', 0)
# feedback.add_correct_char('i', 0)
# feedback.add_correct_char('s', 0)
# feedback.add_correct_char('e', 0)
# feedback.add_wrong_order('a', 0)
# feedback.add_wrong_order('r', 1)
feedback.add_wrong_order('i', 2)
# feedback.add_wrong_order('s', 3)
# feedback.add_wrong_order('e', 4)

# Round 1
possible_answers = get_possible_answers(common_words, feedback)
options = rank_options(possible_answers, feedback)
guess = options[0][0]
print(f'Guess: {guess}')
# feedback.add_incorrect_char('m')
# feedback.add_incorrect_char('i')
feedback.add_correct_char('l', 0)
feedback.add_correct_char('i', 1)
feedback.add_correct_char('t', 4)
# feedback.add_correct_char('m', 4)
# feedback.add_wrong_order('i', 3)
# feedback.add_wrong_order('o', 2)
# feedback.add_wrong_order('c', 3)

# Round 2
possible_answers = get_possible_answers(all_words, feedback)
print('possible_answers')
print(possible_answers)
options = rank_options(possible_answers, feedback)
guess = options[0][0]
print(f'Guess: {guess}')
# # feedback.add_incorrect_char('l')
# # # feedback.add_incorrect_char('l')
# # feedback.add_correct_char('e', 1)
# # feedback.add_correct_char('y', 4)
# # feedback.add_wrong_order('r', 0)
# # feedback.add_wrong_order('p', 2)

# # # Round 3
# possible_answers = get_possible_answers(all_words, feedback)
# options = rank_options(possible_answers, feedback)
# print(options)
# guess = options[0][0]
# print(f'Guess: {guess}')
# feedback.add_correct_char('c', 0)
# feedback.add_correct_char('o', 1)
# feedback.add_incorrect_char('n')

# # Round 4
# possible_answers = get_possible_answers(all_words, feedback)
# options = rank_options(possible_answers, feedback)
# print(options)
# guess = options[0][0]
# print(f'Guess: {guess}')


       civil  fifth  fifty  fight  imply  light  limit  might  night  pilot  \
civil      1      1      1      1      3      1      1      1      1      1   
fifth      6      1      1      1      4      2      3      2      2      2   
fifty      6      1      1      1      1      1      3      1      1      2   
fight      6      1      1      1      4      4      3      4      4      4   
imply      1      1      1      1      1      1      1      1      1      1   
light      2      4      7      4      3      1      1      4      4      2   
limit      1      2      7      2      1      1      1      1      2      2   
might      6      4      7      4      1      4      1      1      4      4   
night      6      4      7      4      4      4      3      4      1      4   
pilot      2      2      7      2      1      1      1      1      2      1   
pitch      1      1      7      1      1      2      3      2      2      1   
tight      6      4      7      4      4      4     

In [9]:
feedback.print()

correct: ['l', 'i', None, None, 't']
incorrect: ['a', 'r', 's', 'e']
wrong order: [['i', 2]]


In [10]:
# # Init game
# feedback = Feedback()
# right_answer = 'query'
# guess = 'arise'
# words_set = all_words

# # Play until the correct answer is found
# turn = 0
# while None in feedback.correct:
#     print('')
#     print('')
#     print('')
#     print('----------------------------------------------------')
#     print(f'Round {turn} - Choice: "{guess}"')
#     print('----------------------------------------------------')
#     print(f'Choice: "{guess}"')
#     # Get feedback
#     feedback = evaluate_guess(guess, right_answer, feedback)
# #     feedback.print()
#     print(feedback.correct)
#     # Get possible answers
#     possible_answers = get_possible_answers(common_words, feedback)
#     if len(possible_answers) == 0:
#         possible_answers = get_possible_answers(all_words, feedback)    
#     print(f'possible answers: {len(possible_answers)}')
#     # Choose next word
#     options = rank_options(possible_answers, feedback)
# #     print(f'ranked options: {options}')
#     # Take the first option
#     guess = options[0][0]
#     turn += 1

